# Carrefour Data Engineer Bootcamp
## Fundamentos de ETL com Python
### Dataset: Ocorrências Aeronáuticas
http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/ocorrencia_2010_2020.csv



### Importando bibliotecas necessárias

In [50]:
import pandas as pd
import pandera as pa

---
# Extração

### Lendo arquivo csv

In [69]:
invalid_data = ("###!", "####", "***", "****", "*****", "NULL")
df = pd.read_csv("ocorrencia_2010_2020.csv", delimiter=";", parse_dates=["ocorrencia_dia"], dayfirst=True, na_values=invalid_data)
df.head()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,12:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO


### Removendo colunas que não serão utilizadas
Parte da limpeza de dados

In [70]:
df.drop(["codigo_ocorrencia1", "codigo_ocorrencia3", "codigo_ocorrencia4", "ocorrencia_latitude", "ocorrencia_longitude", "ocorrencia_pais", "investigacao_aeronave_liberada", "investigacao_status", "divulgacao_relatorio_numero", "divulgacao_relatorio_publicado", "divulgacao_dia_publicacao", "total_aeronaves_envolvidas", "ocorrencia_saida_pista"], inplace=True, axis=1)
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


---
# Validação

In [71]:
df.dtypes
# df.ocorrencia_dia.dt.month

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

### Criando schema de validação

In [76]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia": pa.Column(pa.Int),
        "codigo_ocorrencia2": pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2, 2), nullable=True),
        "ocorrencia_aerodromo": pa.Column(pa.String, nullable=True),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(
            r"^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$"), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int),
    }
)

### Validando DataFrame

In [77]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,NaN,2020-12-31,09:00:00,0


---
# Limpeza

### Substituindo valores inválidos

In [ ]:
# invalid_data = ("###!", "####", "***", "****", "*****", "NULL")
# df.replace(invalid_data, pd.NA, inplace=True)
# movido posteriormente para a célula de leitura do csv (aula de transformaçao)

In [78]:
df.isna().sum()
# df.isnull().sum()  # outra opção

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

---
# Transformação

### Criando coluna ocorrencia_data_hora usando as colunas ocorrencia_dia e ocorrencia_hora

In [79]:
df["ocorrencia_data_hora"] = pd.to_datetime(df.ocorrencia_dia.astype(str) + " " + df.ocorrencia_hora)

In [80]:
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_data_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,2010-01-03 11:05:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0,2010-01-04 17:30:00
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,2010-01-05 19:25:00


### Criando novo dataframe contendo apenas dados de março de 2015

In [82]:
filter_year = df.ocorrencia_data_hora.dt.year == 2015
filter_month = df.ocorrencia_data_hora.dt.month == 3
df201503 = df.loc[filter_year & filter_month]
df201503

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_data_hora
3086,53183,53183,ACIDENTE,AMAPORÃ,PR,NaN,2015-03-02,23:00:00,0,2015-03-02 23:00:00
3087,53120,53120,ACIDENTE,CHAVES,PA,SNXW,2015-03-04,13:30:00,0,2015-03-04 13:30:00
3088,53109,53109,ACIDENTE,CAMPO GRANDE,MS,SSIE,2015-03-05,13:50:00,0,2015-03-05 13:50:00
3089,53112,53112,ACIDENTE,MOGI GUAÇU,SP,NaN,2015-03-06,21:00:00,0,2015-03-06 21:00:00
3090,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
3091,53167,53167,INCIDENTE,MARABÁ,PA,SBMA,2015-03-10,17:33:00,0,2015-03-10 17:33:00
3092,53596,53596,INCIDENTE,ITAPEMA,SC,NaN,2015-03-10,21:35:00,0,2015-03-10 21:35:00
3093,53149,53149,ACIDENTE,TABATINGA,AM,SBTT,2015-03-11,23:25:00,0,2015-03-11 23:25:00
3094,53148,53148,ACIDENTE,ARARAS,SP,SDEH,2015-03-12,12:40:00,0,2015-03-12 12:40:00
3095,53153,53153,INCIDENTE,PORTO ALEGRE,RS,SBPA,2015-03-13,15:45:00,0,2015-03-13 15:45:00


In [84]:
# número de ocorrências de acordo com sua classificação
df201503.groupby(["ocorrencia_classificacao"]).size().sort_values()

ocorrencia_classificacao
INCIDENTE GRAVE     5
ACIDENTE           15
INCIDENTE          17
dtype: int64

### Criando novo dataframe contendo dados da região sudeste no ano de 2010

In [87]:
filter_year = df.ocorrencia_data_hora.dt.year == 2010
filter_region = df.ocorrencia_uf.isin(("MG", "RJ", "SP", "ES"))
dfsoutheast2010 = df.loc[filter_year & filter_region]
dfsoutheast2010

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_data_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0,2010-01-10 20:00:00
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8,2010-01-10 14:50:00
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0,2010-01-15 15:00:00
...,...,...,...,...,...,...,...,...,...,...
541,43722,43722,ACIDENTE,CAJOBI,SP,NaN,2010-12-23,11:00:00,0,2010-12-23 11:00:00
544,43548,43548,INCIDENTE,SÃO PAULO,SP,SBSP,2010-12-26,22:10:00,0,2010-12-26 22:10:00
548,43724,43724,ACIDENTE,COSMÓPOLIS,SP,NaN,2010-12-29,12:00:00,0,2010-12-29 12:00:00
550,43730,43730,INCIDENTE,VITÓRIA,ES,NaN,2010-12-30,18:10:00,0,2010-12-30 18:10:00


In [89]:
# número de ocorrências de acordo com sua classificação
dfsoutheast2010.groupby(["ocorrencia_classificacao"]).size().sort_values()

ocorrencia_classificacao
INCIDENTE GRAVE     18
ACIDENTE            40
INCIDENTE          170
dtype: int64

In [91]:
# número de ocorrências de acordo com sua classificação por estado
dfsoutheast2010.groupby(["ocorrencia_classificacao", "ocorrencia_uf"]).size()

ocorrencia_classificacao  ocorrencia_uf
ACIDENTE                  MG                7
                          RJ                9
                          SP               24
INCIDENTE                 ES                5
                          MG               39
                          RJ               52
                          SP               74
INCIDENTE GRAVE           MG                5
                          RJ                4
                          SP                9
dtype: int64

In [92]:
# número de ocorrências por cidade em ordem decrescente
dfsoutheast2010.groupby(["ocorrencia_cidade"]).size().sort_values(ascending=False)

ocorrencia_cidade
RIO DE JANEIRO           47
SÃO PAULO                33
GUARULHOS                18
BELO HORIZONTE           17
CAMPINAS                 13
MACAÉ                    12
CONFINS                   7
UBERLÂNDIA                5
JUIZ DE FORA              4
VITÓRIA                   4
MARÍLIA                   3
SÃO JOSÉ DOS CAMPOS       3
SÃO JOSÉ DO RIO PRETO     3
SOROCABA                  2
ITANHAÉM                  2
RIBEIRÃO PRETO            2
IPATINGA                  2
POÇOS DE CALDAS           2
DIVINÓPOLIS               2
BAURU                     2
JUNDIAÍ                   2
ARARAQUARA                2
CAMPOS DOS GOYTACAZES     2
AREALVA                   2
MONTES CLAROS             2
UBERABA                   2
POUSO ALEGRE              1
PIRACICABA                1
PIRASSUNUNGA              1
ARRAIAL DO CABO           1
RESENDE                   1
PENÁPOLIS                 1
SANTOS                    1
SERRA NEGRA               1
SÃO BERNARDO DO CAMPO     1
TI

In [93]:
# número de recomendações por aeródromo
dfsoutheast2010.groupby(["ocorrencia_aerodromo"], dropna=False).total_recomendacoes.sum()

ocorrencia_aerodromo
SBAE     0
SBAQ     0
SBBH     0
SBBP     9
SBBU     0
SBCF     0
SBGL     3
SBGR     0
SBIP     0
SBJD     7
SBJR     3
SBKP     0
SBME     0
SBMK     0
SBML     0
SBMT    27
SBPC     0
SBPR     1
SBRJ    19
SBRP     0
SBSJ     0
SBSP     0
SBSR     0
SBUL     0
SBUR     0
SBVT     0
SDAM     0
SDOI     1
SDPW     0
SDPY     0
SNDT     0
SNDV     0
SNXT     0
NaN     45
Name: total_recomendacoes, dtype: int64

In [95]:
# número de recomnedações por cidade (excluindo cidades que não possuem)
filter = dfsoutheast2010.total_recomendacoes > 0
dfsoutheast2010.loc[filter].groupby(["ocorrencia_cidade"]).total_recomendacoes.sum().sort_values()

ocorrencia_cidade
BOITUVA                   1
BOM JESUS DO GALHO        1
ITANHAÉM                  1
BELO HORIZONTE            2
SÃO BERNARDO DO CAMPO     2
IPERÓ                     3
NOVO HORIZONTE            3
TIMÓTEO                   3
JUNDIAÍ                   7
RESENDE                   8
BRAGANÇA PAULISTA         9
CAMANDUCAIA               9
RIO DE JANEIRO           25
SÃO PAULO                41
Name: total_recomendacoes, dtype: int64

In [96]:
# número de recomnedações por cidade e por mês (excluindo cidades que não possuem)
dfsoutheast2010.loc[filter].groupby(["ocorrencia_cidade", dfsoutheast2010.ocorrencia_data_hora.dt.month]).total_recomendacoes.sum()

ocorrencia_cidade      ocorrencia_data_hora
BELO HORIZONTE         2                        1
                       3                        1
BOITUVA                7                        1
BOM JESUS DO GALHO     12                       1
BRAGANÇA PAULISTA      11                       9
CAMANDUCAIA            7                        9
IPERÓ                  1                        3
ITANHAÉM               9                        1
JUNDIAÍ                5                        5
                       6                        2
NOVO HORIZONTE         7                        3
RESENDE                4                        8
RIO DE JANEIRO         4                        3
                       7                        3
                       8                       19
SÃO BERNARDO DO CAMPO  1                        2
SÃO PAULO              1                        8
                       2                       14
                       6                        2
      